# Get Publix locations

#### Load Python tools and Jupyter config

In [1]:
import us 
import json
import black
import requests
import pandas as pd
import jupyter_black
import altair as alt
import geopandas as gpd
from bs4 import BeautifulSoup
from vega_datasets import data
from tqdm.notebook import tqdm, trange

In [ ]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

---

## Scrape

#### Variables we need

In [ ]:
place = "publix"
place_formal = "Publix"
color = "#24670b"
latitude = "39.106667"
longitude = "-94.676392"

#### Headers for requests

In [ ]:
headers = {
    "authority": "services.publix.com",
    "accept": "application/geo+json",
    "accept-language": "en-US,en;q=0.9,es;q=0.8",
    "origin": "https://www.publix.com",
    "referer": "https://www.publix.com/",
    "sec-ch-ua": '"Not A(Brand";v="99", "Google Chrome";v="121", "Chromium";v="121"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"macOS"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
}

#### Make request with huge radius from Tallahassee

In [ ]:
response = requests.get(
    "https://services.publix.com/storelocator/api/v1/stores/?types=R,G,H,N,S&count=5000&distance=5000&zip=32301&isWebsite=true",
    headers=headers,
)

#### Request returns feature collection. Get it into a dataframe

In [ ]:
src = gpd.GeoDataFrame(response.json()["features"])

#### Deal with nested properties column (for store info, address, etc.)

In [ ]:
src[
    [
        "store_number",
        "store_type",
        "store_name",
        "timezone",
        "latitude",
        "longitude",
        "street",
        "city",
        "state",
        "zip",
        "phone",
    ]
] = pd.json_normalize(src["properties"])[
    [
        "storeNumber",
        "type",
        "shortName",
        "timeZone",
        "latitude",
        "longitude",
        "address.streetAddress",
        "address.city",
        "address.state",
        "address.zip",
        "phoneNumbers.Store",
    ]
]

#### Clean up

In [ ]:
df = src.drop(["type", "geometry", "properties"], axis=1).copy()

#### How many locations?

In [ ]:
len(df)

1407

#### Create a mapping of state abbreviations to full state names using the us library

In [10]:
state_mapping = {state.abbr: state.name for state in us.states.STATES}

#### New column of full state names based on abbreviations

In [11]:
df["state_name"] = df["state"].map(state_mapping)

---

## Geography

#### Make it a geodataframe

In [12]:
df_geo = df.copy()

In [13]:
gdf = gpd.GeoDataFrame(
    df_geo, geometry=gpd.points_from_xy(df_geo.longitude, df_geo.latitude)
)

---

## Maps

#### US states background

In [14]:
background = (
    alt.Chart(alt.topo_feature(data.us_10m.url, feature="states"))
    .mark_geoshape(fill="#e9e9e9", stroke="white")
    .properties(width=800, height=500, title=f"{place_formal} locations")
    .project("albersUsa")
)

#### Location points map

In [15]:
points = (
    alt.Chart(gdf)
    .mark_circle(size=5, color=color)
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
    )
)

point_map = background + points
point_map.configure_view(stroke=None)

alt.LayerChart(...)

#### Location proportional symbols map

In [16]:
symbols = (
    alt.Chart(gdf)
    .transform_aggregate(
        latitude="mean(latitude)",
        longitude="mean(longitude)",
        count="count()",
        groupby=["state"],
    )
    .mark_circle()
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
        size=alt.Size("count:Q", title="Count by state"),
        color=alt.value(color),
        tooltip=["state:N", "count:Q"],
    )
    .properties(
        title=f"Number of {place_formal} in US, by average lon/lat of locations"
    )
)

symbol_map = background + symbols
symbol_map.configure_view(stroke=None)

alt.LayerChart(...)

---

## Exports

#### JSON

In [18]:
df.to_json(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.json",
    indent=4,
    orient="records",
)

#### CSV

In [19]:
df.to_csv(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.csv", index=False
)

#### GeoJSON

In [21]:
gdf.to_file(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.geojson",
    driver="GeoJSON",
)